In [1]:
# ============================================
# Notebook: test_pyspark.ipynb
# Objetivo: Testar instalação do PySpark
# Data: 15/10/2025
# ============================================

# CÉLULA 1: Configurar ambiente e importar bibliotecas
import os
import sys

# FIX: Garantir que Spark use o mesmo Python do Jupyter
python_path = sys.executable
os.environ['PYSPARK_PYTHON'] = python_path
os.environ['PYSPARK_DRIVER_PYTHON'] = python_path
os.environ['SPARK_LOCAL_HOSTNAME'] = 'localhost'

print("=== CONFIGURAÇÃO DO AMBIENTE ===")
print(f"✓ Python: {sys.version}")
print(f"✓ Executável: {python_path}")
print(f"✓ PYSPARK_PYTHON: {os.environ.get('PYSPARK_PYTHON')}")
print(f"✓ SPARK_LOCAL_HOSTNAME: {os.environ.get('SPARK_LOCAL_HOSTNAME')}")
print("="*60)

print("\n=== IMPORTANDO BIBLIOTECAS ===")
from pyspark.sql import SparkSession
import pandas as pd
print("✓ Bibliotecas importadas com sucesso!")

# ============================================

# CÉLULA 2: Criar Spark Session
print("\n=== CRIANDO SPARK SESSION ===")

# Configuração especial para Windows com hostname contendo underscore
import os
os.environ['SPARK_LOCAL_HOSTNAME'] = 'localhost'

spark = SparkSession.builder \
    .appName("TestePySpark") \
    .master("local[*]") \
    .config("spark.driver.memory", "2g") \
    .config("spark.driver.host", "localhost") \
    .config("spark.driver.bindAddress", "127.0.0.1") \
    .getOrCreate()

print(f"✓ Spark Session criada!")
print(f"✓ Versão do Spark: {spark.version}")
print(f"✓ App Name: {spark.sparkContext.appName}")

# ============================================

# CÉLULA 3: Criar DataFrame dummy em memória
print("\n=== CRIANDO DATAFRAME DUMMY ===")

# Dados de exemplo
dados = [
    (1, "João Silva", 28, "São Paulo", 5500.00),
    (2, "Maria Santos", 34, "Rio de Janeiro", 6200.00),
    (3, "Pedro Oliveira", 25, "Belo Horizonte", 4800.00),
    (4, "Ana Costa", 41, "Curitiba", 7100.00),
    (5, "Carlos Souza", 29, "Porto Alegre", 5200.00),
    (6, "Julia Mendes", 37, "Brasília", 6800.00),
    (7, "Lucas Ferreira", 31, "Fortaleza", 5400.00),
    (8, "Fernanda Lima", 26, "Salvador", 4900.00),
]

# Definir schema (colunas)
colunas = ["id", "nome", "idade", "cidade", "salario"]

# Criar DataFrame Spark
df_spark = spark.createDataFrame(dados, colunas)

print("✓ DataFrame criado com sucesso!")
print(f"✓ Total de registros: {df_spark.count()}")
print(f"✓ Colunas: {df_spark.columns}")

# ============================================

# CÉLULA 4: Visualizar dados
print("\n=== VISUALIZANDO DADOS ===")
df_spark.show()

# ============================================

# CÉLULA 5: Estatísticas básicas
print("\n=== ESTATÍSTICAS DESCRITIVAS ===")
df_spark.describe().show()

# ============================================

# CÉLULA 6: Criar CSV para teste de leitura
print("\n=== CRIANDO ARQUIVO CSV PARA TESTE ===")

# Criar diretório data se não existir
import os
os.makedirs("../data", exist_ok=True)

# Salvar DataFrame como CSV
csv_path = "../data/funcionarios_teste.csv"
df_spark.coalesce(1).write.mode("overwrite").csv(csv_path, header=True)

print(f"✓ CSV salvo em: {csv_path}")

# ============================================

# CÉLULA 7: Testar spark.read.csv()
print("\n=== TESTANDO SPARK.READ.CSV() ===")

# Ler CSV com Spark
df_lido = spark.read.csv(
    csv_path,
    header=True,
    inferSchema=True
)

print("✓ CSV lido com sucesso!")
print(f"✓ Total de registros lidos: {df_lido.count()}")

# Mostrar dados lidos
print("\n=== DADOS LIDOS DO CSV ===")
df_lido.show()

# ============================================

# CÉLULA 8: Operações básicas com Spark
print("\n=== OPERAÇÕES BÁSICAS COM SPARK ===")

# 1. Filtrar por idade
print("\n1. Funcionários com mais de 30 anos:")
df_lido.filter(df_lido.idade > 30).show()

# 2. Selecionar colunas específicas
print("\n2. Apenas nome e salário:")
df_lido.select("nome", "salario").show()

# 3. Ordenar por salário
print("\n3. Ordenado por salário (decrescente):")
df_lido.orderBy(df_lido.salario.desc()).show()

# 4. Agrupar por cidade e calcular média salarial
print("\n4. Salário médio por cidade:")
df_lido.groupBy("cidade").avg("salario").show()

# ============================================

# CÉLULA 9: Converter para Pandas (para análise mais detalhada)
print("\n=== CONVERTENDO PARA PANDAS ===")

df_pandas = df_lido.toPandas()
print("✓ Convertido para Pandas!")
print(f"✓ Tipo: {type(df_pandas)}")
print("\nPrimeiras linhas:")
print(df_pandas.head())

# ============================================

# CÉLULA 10: Schema do DataFrame
print("\n=== SCHEMA DO DATAFRAME ===")
df_lido.printSchema()

# ============================================

# CÉLULA 11: Informações do Spark Context
print("\n=== INFORMAÇÕES DO SPARK CONTEXT ===")
print(f"Master: {spark.sparkContext.master}")
print(f"Default Parallelism: {spark.sparkContext.defaultParallelism}")
print(f"Spark User: {spark.sparkContext.sparkUser()}")

# ============================================

# CÉLULA 12: SQL com Spark
print("\n=== TESTANDO SPARK SQL ===")

# Criar view temporária
df_lido.createOrReplaceTempView("funcionarios")

# Executar query SQL
resultado_sql = spark.sql("""
    SELECT 
        cidade,
        COUNT(*) as total_funcionarios,
        AVG(salario) as salario_medio,
        MAX(salario) as maior_salario
    FROM funcionarios
    GROUP BY cidade
    ORDER BY salario_medio DESC
""")

print("✓ Query SQL executada!")
resultado_sql.show()

# ============================================

# CÉLULA 13: Finalização
print("\n=== TESTE COMPLETO ===")
print("✓ PySpark instalado e funcionando!")
print("✓ Jupyter Notebook integrado com sucesso!")
print("✓ spark.read.csv() funcionando perfeitamente!")
print("✓ Operações básicas testadas!")
print("\n🎉 TODOS OS OBJETIVOS CUMPRIDOS! 🎉")

# Não fechar a sessão ainda (para continuar usando o notebook)
# spark.stop()  # Descomente para encerrar

=== CONFIGURAÇÃO DO AMBIENTE ===
✓ Python: 3.10.0 (tags/v3.10.0:b494f59, Oct  4 2021, 19:00:18) [MSC v.1929 64 bit (AMD64)]
✓ Executável: C:\Program Files\Python310\python.exe
✓ PYSPARK_PYTHON: C:\Program Files\Python310\python.exe
✓ SPARK_LOCAL_HOSTNAME: localhost

=== IMPORTANDO BIBLIOTECAS ===
✓ Bibliotecas importadas com sucesso!

=== CRIANDO SPARK SESSION ===
✓ Spark Session criada!
✓ Versão do Spark: 4.0.1
✓ App Name: TestePySpark

=== CRIANDO DATAFRAME DUMMY ===
✓ DataFrame criado com sucesso!
✓ Total de registros: 8
✓ Colunas: ['id', 'nome', 'idade', 'cidade', 'salario']

=== VISUALIZANDO DADOS ===
+---+--------------+-----+--------------+-------+
| id|          nome|idade|        cidade|salario|
+---+--------------+-----+--------------+-------+
|  1|    João Silva|   28|     São Paulo| 5500.0|
|  2|  Maria Santos|   34|Rio de Janeiro| 6200.0|
|  3|Pedro Oliveira|   25|Belo Horizonte| 4800.0|
|  4|     Ana Costa|   41|      Curitiba| 7100.0|
|  5|  Carlos Souza|   29|  Porto Al